In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow

In [2]:
import pandas as pd
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.metrics import mean_squared_error

sns.set_style('whitegrid')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
series_new = read_csv('../data/clean_df.csv', header=0, parse_dates=[2], index_col=0, squeeze=True)

In [ ]:
error_list = []

for dataid in series_new['dataid'].unique():
    
    data = series_new.sort_values(['dataid','localhour'])[(series_new['dataid']==dataid) 
                                                          & (series_new['localhour'] < '2015-04-02')
                                                          & (series_new['localhour'] >= '2015-03-12')]

    data['udiff'] = data['usage'].diff()
    data.dropna(inplace=True)

    urate = data[['udiff']].values

    #mms = StandardScaler()
    #urate = mms.fit_transform(urate)

    train_size = int(len(urate) * 0.95)
    test_size = len(urate) - train_size

    train, test = urate[0:train_size,:], urate[train_size:len(urate),:]

    udiff = data.udiff

    udiff = udiff[0:8]

    pd.DataFrame(dict(y=udiff.shift(1)))

    def create_data(timeseries, lag=1, as_array=True):
        if not isinstance(timeseries, pd.Series):
            timeseries = pd.Series(timeseries.ravel())
        y = timeseries[lag:]
        X = pd.DataFrame({'lag'+str(lag-i):timeseries.shift(-i) for i in range(0, lag)}).dropna().iloc[:-1, :]
        if not as_array:
            return X, y
        else:
            return X.values, y.values

    X, y = create_data(udiff, lag=3)

    lag = 1
    trainX, trainY = create_data(train, lag)
    testX, testY = create_data(test, lag)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    batch_size = 1
    model = Sequential()
    model.add(LSTM(4, batch_input_shape=(batch_size, lag, 1), stateful=True))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(7):
        model.fit(trainX, trainY, nb_epoch=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()

    trainPredict = model.predict(trainX, batch_size=batch_size)
    model.reset_states()
    testPredict = model.predict(testX, batch_size=batch_size)
    model.reset_states()

    error = mean_squared_error(urate[-25:], testPredict)
    error_list.append(error)
    print("Ave error: " + str(sum(error_list)/len(error_list)) + ", dataid: " + str(dataid) + ", error: " + str(error))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/opt/conda/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
476/476 [==============================] - 2s - loss: 0.4664     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.4417     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.4218     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.4013     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.3838     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.3726     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.3654     
Ave error: 1.01069742304, dataid: 26, error: 1.01069742304
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.8054     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.7704     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.7394     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.7128     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.6906     
Epoch 1/1
476/476 [==========

476/476 [==============================] - 2s - loss: 0.6242     
Ave error: 0.473359855322, dataid: 434, error: 1.0166331609
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.0626     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.0597     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.0566     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.0537     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.0518     
Epoch 1/1
476/476 [==============================] - 1s - loss: 0.0513     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.0508     
Ave error: 0.447576220881, dataid: 484, error: 0.0866053387057
Epoch 1/1
476/476 [==============================] - 3s - loss: 0.2451     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.2311     
Epoch 1/1
476/476 [==============================] - 3s - loss: 0.2233     
Epoch 1/1
476/476 [==============================] 

476/476 [==============================] - 2s - loss: 0.0961     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.0948     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.0935     
Ave error: 0.462206421851, dataid: 821, error: 0.0566859599422
Epoch 1/1
476/476 [==============================] - 3s - loss: 0.5558     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.5302     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.5171     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.5082     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.5000     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.4929     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.4871     
Ave error: 0.455549651914, dataid: 871, error: 0.269160093685
Epoch 1/1
476/476 [==============================] - 6s - loss: 0.1409     
Epoch 1/1
476/476 [==============================

476/476 [==============================] - 2s - loss: 0.0915     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.0894     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.0862     
Epoch 1/1
476/476 [==============================] - 7s - loss: 0.0849     
Epoch 1/1
476/476 [==============================] - 3s - loss: 0.0817     
Ave error: 0.388760960733, dataid: 1403, error: 0.0785406048654
Epoch 1/1
476/476 [==============================] - 4s - loss: 0.3815     
Epoch 1/1
476/476 [==============================] - 3s - loss: 0.3590     
Epoch 1/1
476/476 [==============================] - 3s - loss: 0.3420     
Epoch 1/1
476/476 [==============================] - 4s - loss: 0.3300     
Epoch 1/1
476/476 [==============================] - 7s - loss: 0.3219     
Epoch 1/1
476/476 [==============================] - 4s - loss: 0.3167     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.3136     
Ave error: 0.381457134045, dataid:

476/476 [==============================] - 424s - loss: 0.2649   
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.2468     
Epoch 1/1
476/476 [==============================] - -858s - loss: 0.2305  
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.2234     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.2208     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.2192     
Epoch 1/1
476/476 [==============================] - 4s - loss: 0.2188     
Ave error: 0.410858023217, dataid: 1766, error: 0.0901217476197
Epoch 1/1
476/476 [==============================] - 74s - loss: 0.5492    
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.5372     
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.5262     
Epoch 1/1
476/476 [==============================] - 43s - loss: 0.5169    
Epoch 1/1
476/476 [==============================] - 2s - loss: 0.5092     
Epoch 1/1
476/476 [===============

476/476 [==============================] - 3s - loss: 0.2516     
Epoch 1/1
476/476 [==============================] - 3s - loss: 0.2487     
Ave error: 0.389541394807, dataid: 2129, error: 0.199969896927
Epoch 1/1
476/476 [==============================] - 8s - loss: 1.4142     
Epoch 1/1
476/476 [==============================] - 5s - loss: 1.3438     
Epoch 1/1
476/476 [==============================] - 4s - loss: 1.2808     
Epoch 1/1
476/476 [==============================] - 6s - loss: 1.2275     
Epoch 1/1
476/476 [==============================] - 5s - loss: 1.1956     
Epoch 1/1
476/476 [==============================] - 3s - loss: 1.1793     
Epoch 1/1
476/476 [==============================] - 3s - loss: 1.1689     
Ave error: 0.39382649428, dataid: 2156, error: 0.689498357859
Epoch 1/1
476/476 [==============================] - 8s - loss: 0.7362     
Epoch 1/1
476/476 [==============================] - 4s - loss: 0.7072     
Epoch 1/1
476/476 [==============================

476/476 [==============================] - 8s - loss: 0.0823     
Epoch 1/1
476/476 [==============================] - 6s - loss: 0.0806     
Epoch 1/1
476/476 [==============================] - 7s - loss: 0.0792     
Epoch 1/1
476/476 [==============================] - 8s - loss: 0.0782     
Ave error: 0.419726100799, dataid: 2557, error: 0.0485277204994
Epoch 1/1
476/476 [==============================] - 13s - loss: 0.4401    
Epoch 1/1
476/476 [==============================] - 8s - loss: 0.4278     
Epoch 1/1
476/476 [==============================] - 10s - loss: 0.4160    
Epoch 1/1
476/476 [==============================] - 10s - loss: 0.4060    
Epoch 1/1
476/476 [==============================] - 7s - loss: 0.3996     
Epoch 1/1
476/476 [==============================] - 6s - loss: 0.3957     
Epoch 1/1
476/476 [==============================] - 5s - loss: 0.3927     
Ave error: 0.41686845339, dataid: 2575, error: 0.179683718457
Epoch 1/1
476/476 [=============================

In [ ]:
error = mean_squared_error(urate[-25:], testPredict)

In [ ]:
# plot baseline and predictions
plt.figure(figsize=(14,7))
plt.plot(urate[-25:])
#plt.plot(trainPredictPlot)
plt.plot(testPredict, color="red")
plt.xlabel("Hour of the Month")
plt.ylabel("Electricity Usage (kWh)")
plt.title("LSTM Usage Prediction - 4/1/2015")
plt.legend()
plt.show()


In [ ]:
dataid